<a href="https://colab.research.google.com/github/sefeoglu/AE_Parseval_Network/blob/master/src/notebooks/FGSM_WResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import sys
sys.path.insert(0, os.path.abspath('../'))

In [0]:
from sklearn.model_selection import train_test_split
import gzip
import pickle
import numpy as np

In [159]:
def read_data():
    with open("data.pz", 'rb') as file_:
        with gzip.GzipFile(fileobj=file_) as gzf:
            data = pickle.load(gzf, encoding='latin1', fix_imports=True)
    return data
data = read_data()
new_data_X = []
Y_data = []
for row in data:
    new_data_X.append(row['crop'])
    Y_data.append(row['label'])
new_data_X = np.array(new_data_X)
new_data_X.shape

(5722, 68, 100)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(new_data_X, Y_data, test_size=0.33, random_state=42)

In [0]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
# creating initial dataframe

y_train_df = pd.DataFrame(y_train, columns=['Label'])
# creating instance of labelencoder
labelencoder = LabelEncoder()
# Assigning numerical values and storing in another column
y_train_df['New'] = labelencoder.fit_transform(y_train_df['Label'])
y_test_df = pd.DataFrame(y_test, columns=['Label'])
# creating instance of labelencoder
labelencoder = LabelEncoder()
# Assigning numerical values and storing in another column
y_test_df['New'] = labelencoder.fit_transform(y_test_df['Label'])

In [0]:
import keras.backend as K
img_rows, img_cols = X_train[0].shape


# transform data set
if K.common.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [0]:
from keras.utils import to_categorical
y_test = to_categorical(y_test_df['New'])
y_train = to_categorical(y_train_df['New'])

In [0]:
import keras

model = tf.keras.models.load_model("wresnet.h5")

Adversarial Attack


In [166]:
model.input.shape

TensorShape([None, 68, 100, 1])

In [0]:
def preprocess(image):
  image = tf.cast(image, tf.float32)
  image = tf.image.resize(image, (68, 100))
  image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
  image = image[None, ...]
  return image

# Helper function to extract labels from probability vector

In [168]:
img = X_train[8]
img.shape

(68, 100, 1)

In [0]:
vgg19 = tf.keras.applications.VGG19()

In [181]:
features_list = [layer.output for layer in model.layers]
features_list

[<tf.Tensor 'input_1_5:0' shape=(None, 68, 100, 1) dtype=float32>,
 <tf.Tensor 'conv2d_1_4/Identity:0' shape=(None, 68, 100, 16) dtype=float32>,
 <tf.Tensor 'batch_normalization_1_4/Identity:0' shape=(None, 68, 100, 16) dtype=float32>,
 <tf.Tensor 'activation_1_5/Identity:0' shape=(None, 68, 100, 16) dtype=float32>,
 <tf.Tensor 'conv2d_2_4/Identity:0' shape=(None, 68, 100, 32) dtype=float32>,
 <tf.Tensor 'batch_normalization_2_4/Identity:0' shape=(None, 68, 100, 32) dtype=float32>,
 <tf.Tensor 'activation_2_5/Identity:0' shape=(None, 68, 100, 32) dtype=float32>,
 <tf.Tensor 'conv2d_3_4/Identity:0' shape=(None, 68, 100, 32) dtype=float32>,
 <tf.Tensor 'conv2d_4_4/Identity:0' shape=(None, 68, 100, 32) dtype=float32>,
 <tf.Tensor 'add_1_5/Identity:0' shape=(None, 68, 100, 32) dtype=float32>,
 <tf.Tensor 'batch_normalization_3_4/Identity:0' shape=(None, 68, 100, 32) dtype=float32>,
 <tf.Tensor 'activation_3_5/Identity:0' shape=(None, 68, 100, 32) dtype=float32>,
 <tf.Tensor 'conv2d_5_4/Ide

In [187]:
feat_extraction_model = tf.keras.models.Model(inputs=model.input, outputs=model.layers[-1].output)
feat_extraction_model

In [172]:
import tensorflow as tf
image = tf.convert_to_tensor(img)
image = preprocess(image)
image.shape

TensorShape([1, 68, 100, 1])

In [188]:
feat_extraction_model(image)




<tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[0.02893857, 0.2202906 , 0.09846022, 0.6523106 ]], dtype=float32)>

In [189]:
probs = model.predict(image,batch_size=None,steps=1)

label = tf.one_hot(3, probs.shape[-1])
label = tf.reshape(label, (1, probs.shape[-1]))
label

<tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[0., 0., 0., 1.]], dtype=float32)>

In [0]:
loss_object = tf.keras.losses.CategoricalCrossentropy()

def create_adversarial_pattern(input_image, input_label):
  with tf.GradientTape() as tape:
    tape.watch(input_image)
    prediction = model(input_image)
    loss = loss_object(input_label, prediction)

  # Get the gradients of the loss w.r.t to the input image.
  gradient = tape.gradient(loss, input_image)

  # Get the sign of the gradients to create the perturbation
  signed_grad = tf.sign(gradient)
  return signed_grad

In [202]:
import matplotlib.pyplot as plt
perturbations = create_adversarial_pattern(image,label)
adve = perturbations+ 0.3*image
adve

tf.Tensor(
[[[[-2.59939494e-04]
   [-6.70405047e-04]
   [-8.01580085e-04]
   ...
   [-1.32049248e-03]
   [-7.17259711e-04]
   [-3.90303787e-04]]

  [[-7.80968054e-04]
   [-1.79722940e-03]
   [-2.20469385e-03]
   ...
   [-4.75630816e-03]
   [-2.38177879e-03]
   [-1.26672606e-03]]

  [[-6.57051860e-04]
   [-1.69656775e-03]
   [-1.71961659e-03]
   ...
   [-4.91720578e-03]
   [-1.17614213e-03]
   [-6.98767544e-04]]

  ...

  [[ 2.43612194e-05]
   [ 9.29113245e-04]
   [ 2.11995817e-03]
   ...
   [ 1.75066976e-04]
   [ 2.06759156e-04]
   [ 1.54479669e-04]]

  [[ 3.05417692e-04]
   [ 7.75912777e-04]
   [ 8.57825624e-04]
   ...
   [ 2.81883476e-05]
   [ 1.63886871e-04]
   [ 1.34231435e-04]]

  [[ 1.29242035e-04]
   [ 7.40479954e-05]
   [-3.18969396e-04]
   ...
   [ 1.91707906e-04]
   [ 1.66031896e-04]
   [ 1.05009465e-04]]]], shape=(1, 68, 100, 1), dtype=float32)


<tf.Tensor: shape=(1, 68, 100, 1), dtype=float32, numpy=
array([[[[-1.300999  ],
         [-1.300999  ],
         [-1.3012393 ],
         ...,
         [-1.2983574 ],
         [-1.2985976 ],
         [-1.2997983 ]],

        [[-1.3005188 ],
         [-1.3014793 ],
         [-1.3012393 ],
         ...,
         [-1.2978771 ],
         [-1.2981173 ],
         [-1.2993181 ]],

        [[-1.3014793 ],
         [-1.300999  ],
         [-1.3012393 ],
         ...,
         [-1.2973968 ],
         [-1.2985976 ],
         [-1.2993181 ]],

        ...,

        [[ 0.6980403 ],
         [ 0.69780016],
         [ 0.69780016],
         ...,
         [ 0.7035638 ],
         [ 0.703804  ],
         [ 0.7030835 ]],

        [[ 0.69780016],
         [ 0.69780016],
         [ 0.69756   ],
         ...,
         [ 0.7033237 ],
         [ 0.703804  ],
         [ 0.7033237 ]],

        [[ 0.69780016],
         [ 0.69828045],
         [-1.3024399 ],
         ...,
         [ 0.7035638 ],
         [ 0.704284

In [205]:
np.argmax(model.predict(adve))

1

In [0]:
from six.moves import xrange
red_ind = list(xrange(1, len(image.get_shape())))
square = tf.reduce_sum(tf.square(grad))
normalized_grad = grad / tf.sqrt(square)
normalized_grad

In [0]:
scaled_grad = 0.3 * normalized_grad
adv = scaled_grad+image
tf.compat

In [0]:
probs = tf.convert_to_tensor(probs)

In [0]:
probs

In [0]:
 l = keras.losses.categorical_crossentropy(label, probs)

In [0]:
print(l)

In [0]:
loss_object = keras.losses.CategoricalCrossentropy()
variable = tf.Variable(image)
np.gradient(np.array([l]))

In [0]:
generate_pattern("dense_")
